In [1]:
import threading
import requests
import queue
import requests
import csv
import re
from bs4 import BeautifulSoup
import sys
import time
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import random

In [2]:
def read_proxies_from_file(file_path):
    """
    Read proxies from a text file.
    """
    with open(file_path, 'r') as file:
        proxies = [line.strip() for line in file]
    return proxies

In [3]:
def check_proxy(proxy):
    """
    Check if a proxy is working by making a test request.
    """
    try:
        response = requests.get('https://magicpin.in/Mumbai/Kurla/Restaurant/Fast-N%E2%80%99-Fresh/store/1557c6b/delivery/', proxies={'http': proxy, 'https': proxy}, timeout=5)
        return response.status_code == 200
    except Exception as e:
        return False

In [4]:
def get_random_working_proxy(proxies):
    """
    Get a random working proxy from the list of proxies.
    """
    random.shuffle(proxies)
    for proxy in proxies:
        if check_proxy(proxy):
            return proxy
    return None

In [ ]:
def scrape_and_save_data(url, output_csv, proxies):
    # Set up Selenium WebDriver with a random proxy
    proxy = get_random_working_proxy(proxies)
    if not proxy:
        print("No working proxy found.")
        return
    # Specify the path for the CSV file
    csv_file_path = 'scraped.csv'
    
    # Use Edge WebDriver
    driver_path = 'C:/Users/Khushi/Documents/Cooltrack/edgedriver_win64/msedgedriver.exe'
    options = webdriver.EdgeOptions()
    #options.use_chromium = True
    options.add_argument('--proxy-server={}'.format(proxy))
    driver = webdriver.Edge(executable_path=driver_path, options=options)

    try:
        # Navigate to the webpage
        driver.get(url)

        # Find all <span> elements within <p> elements with class "listedItemDetails"
        span_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'p.listedItemDetails span'))
        )

        # Click on each <span> element
        for span_element in span_elements:
            span_element.click()

        # Get the updated HTML content
        updated_html = driver.page_source

        # Parse the updated HTML with BeautifulSoup
        soup = BeautifulSoup(updated_html, 'html.parser')

        # Initialize a list to store the data
        data_list = []

        # For example, find all elements with class 'itemInfo'
        item_infos = soup.find_all('article', class_='itemInfo')

        for item_info in item_infos:
            item_name = item_info.find('p', class_='itemName').a.text.strip()
            item_price = item_info.find('span', class_='itemPrice').text.strip()

            # Save item_name and item_price in the list
            data_list.append({'Item Name': item_name, 'Item Price': item_price})

        # Replace the currency symbol in the 'Item Price' field
        for item in data_list:
            if '₹' in item['Item Price']:
                item['Item Price'] = 'Rs.' + item['Item Price'][1:]

        # Save the list to a CSV file
        with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
            fieldnames = ['Item Name', 'Item Price']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            # Write the header
            writer.writeheader()

            # Write the data
            writer.writerows(data_list)

        print(f'List saved to CSV file: {output_csv}')

    finally:
        # Close the browser window
        driver.quit()

In [ ]:
def main():
    
    # Specify the list of websites to scrape
    websites = [
    {'url': 'https://magicpin.in/Mumbai/Kurla/Restaurant/Fast-N%E2%80%99-Fresh/store/1557c6b/delivery/', 'output_csv': 'output1.csv'},
    {'url': 'https://magicpin.in/Mumbai/Chembur/Restaurant/Rare-Kitchen-Friends-and-Family/store/1553893/delivery/', 'output_csv': 'output2.csv'},
    {'url': 'https://magicpin.in/Mumbai/Govandi-West/Restaurant/Ss-Global-Chinese-Hut/store/1593116/delivery/', 'output_csv': 'output3.csv'},
    # Add more websites as needed
    ]

    # Specify the path to the text file containing proxies
    proxy_file_path = 'proxy_list.txt'
    proxies = read_proxies_from_file(proxy_file_path)

    # Scrape data for each website with a random proxy
    for website in websites:
        scrape_and_save_data(website['url'], website['output_csv'], proxies)
        
if __name__ == "__main__":
    main()